In [27]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 

In [14]:
from sklearn.model_selection import StratifiedKFold
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

#y è array di array, io voglio array!
X = np.array(X)
y = np.array(y)
X=X.astype(float)
Y=y.astype(float)
y=np.where(y==0,-1,y)

In [15]:
#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
#for train_index, test_index in skf.split(X, y):
#    #print("TRAIN:", train_index, "\nTEST:", test_index)
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]

In [16]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=90.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,len(y_test)):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,len(y_predict)):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
             pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

In [17]:
from cvxopt import matrix
class HYP_SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        print(self.m.shape)
        
 ##############################################################################


    def fit(self, X_train,X_test, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [29]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        clf = HYP_SVM(C=100.0)
        typ=2
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
    
            clf.m_func(X_train,X_test,y_train)
            clf.fit(X_train,X_test, y_train)
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
            
    hyp_svm()    

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -4.4850e+03 -1.5659e+05  2e+05  6e-13  3e-13
 1: -4.5794e+03 -1.0051e+04  5e+03  2e-12  5e-13
 2: -5.3482e+03 -9.2682e+03  4e+03  4e-13  4e-13
 3: -6.0886e+03 -8.3477e+03  2e+03  8e-13  5e-13
 4: -6.6199e+03 -7.4566e+03  8e+02  2e-13  6e-13
 5: -6.8005e+03 -7.2157e+03  4e+02  7e-13  5e-13
 6: -6.8692e+03 -7.1261e+03  3e+02  2e-12  5e-13
 7: -6.9268e+03 -7.0556e+03  1e+02  4e-13  5e-13
 8: -6.9595e+03 -7.0159e+03  6e+01  2e-16  6e-13
 9: -6.9812e+03 -6.9904e+03  9e+00  2e-12  7e-13
10: -6.9855e+03 -6.9856e+03  1e-01  2e-12  7e-13
11: -6.9855e+03 -6.9855e+03  1e-03  2e-12  7e-13
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -4.4850e+03 -1.5659e+05  2e+05  6e-13  3e-13
 1: -4.5794e+03 -1.0051e+04  5e+03  2e-12  5e-13
 2: -5.3482e+03 -9.2682e+03  4e+03  4e-13  4e-13
 3: -6.0886e+03 -8.3477e+03  2e+03  8e-13  5e-13
 4: -6.6199e+03 -7.4566e+03  8e+02  2e-13  6e-13
 5:

In [19]:
clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

Overall RBF KERNEL SVM accuracy:  1.0


In [31]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [32]:
if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)    
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
    normal_svm()   

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -4.4850e+03 -1.5659e+05  2e+05  6e-13  3e-13
 1: -4.5794e+03 -1.0051e+04  5e+03  2e-12  5e-13
 2: -5.3482e+03 -9.2682e+03  4e+03  4e-13  4e-13
 3: -6.0886e+03 -8.3477e+03  2e+03  8e-13  5e-13
 4: -6.6199e+03 -7.4566e+03  8e+02  2e-13  6e-13
 5: -6.8005e+03 -7.2157e+03  4e+02  7e-13  5e-13
 6: -6.8692e+03 -7.1261e+03  3e+02  2e-12  5e-13
 7: -6.9268e+03 -7.0556e+03  1e+02  4e-13  5e-13
 8: -6.9595e+03 -7.0159e+03  6e+01  2e-16  6e-13
 9: -6.9812e+03 -6.9904e+03  9e+00  2e-12  7e-13
10: -6.9855e+03 -6.9856e+03  1e-01  2e-12  7e-13
11: -6.9855e+03 -6.9855e+03  1e-03  2e-12  7e-13
Optimal solution found.
optimal
(120,)
118 support vectors out of 120 points
y_test min 10
y_test max 20
y_pred min 7
y_pred max 20
0.7 1.0
GM 0.8366600265340756
27 out of 30 predictions correct
Accuracy 0.9
Errore quadratico medio:  0.4
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -4.6173e+03 -1.5693e+05  2e+05  8e-13  4e-13
 1: -4.